# Take the SM1 and SM2 data and split them into different files

In [2]:
import re as regex

In [3]:
# function to get row
getrow = lambda x: int(regex.search("(?<=CCD_ROW_)[0-9]+",x).group(0))
# function to get OBMT
getobmt = lambda x: int(regex.search("(?<=OBMT_START_)[0-9]+",x).group(0))

In [4]:
# construct a list like a numpy array, for each fov and row

filenames = [[],[]]

for ii in range(7):
    filenames[0].append([])
    filenames[1].append([])
    
# access structure is filenames[fov-1][row-1]

for fov in [1,2]:

    pathfile = "../../filenames_sm{}.txt".format(fov)


    allfiles = []
    f = open(pathfile, 'r')
    allfiles = f.readlines()
    f.close()

    for file in allfiles:
        row = getrow(file)
        filenames[fov-1][row-1].append(file)

In [5]:
# now we need to weed out all the calibration activities
# for each row, I will give a tuple of OBMT_START and END of regions to exclude

# CI_IN_SM - this applies to ALL rows

OBMT_CI = [(6.81042889094578E+015,6.81522889094786E+015),
           (1.78191982102836E+016,1.78233982102849E+016),
           (4.29615982403429E+016,4.29657982403473E+016),
           (5.44455992558824E+016,5.44539992558934E+016),
           (6.01479992631641E+016,6.01563992631747E+016),
           (7.0170399276483E+016,7.01787992764946E+016),
           (8.07111992907961E+016,8.0719599290808E+016),
           (8.78175993010565E+016,8.78259993010686E+016),
           (9.85959993166397E+016,9.86001993166458E+016),
           (1.10173600334343E+017,1.10178400334351E+017),
           (1.2062800035135E+017,1.20632200351357E+017)
          ]

# FVBS - Forward bias, row-wise

FVBS1 = [(6.70902889090192E+015,6.71532889090465E+015),
         (1.36233982073171E+016,1.36305982073196E+016),
         (5.11626982515722E+016,5.11634182515731E+016),
         (9.24618993078481E+016,9.2467299307856E+016),
         (1.1967790034981E+017,1.19683300348819E+017)
        ]

FVBS2 = [(6.70902889090192E+015,6.71532889090465E+015),
         (1.36233982073171E+016,1.36305982073196E+016),
         (5.13354982517898E+016,5.13362182517907E+016),
         (9.24618993078481E+016,9.2467299307856E+016),
         (1.1967790034981E+017,1.19683300348819E+017)
        ]

FVBS3 = [(6.70902889090192E+015,6.71532889090465E+015),
         (1.36233982073171E+016,1.36305982073196E+016),
         (5.15082982520074E+016,5.15090182520083E+016),
         (9.24618993078481E+016,9.2467299307856E+016),
         (1.1967790034981E+017,1.19683300348819E+017)
        ]

FVBS4 = [(6.70902889090192E+015,6.71532889090465E+015),
         (1.36233982073171E+016,1.36305982073196E+016),
         (5.1681098252225E+016,5.16818182522259E+016),
         (9.27210993082233E+016,9.27282993082337E+016),
         (1.19429200349407E+017,1.19436400349418E+017)
        ]

FVBS5 = [(6.70902889090192E+015,6.71532889090465E+015),
         (1.36233982073171E+016,1.36305982073196E+016),
         (5.18538982524426E+016,5.18546182524435E+016),
         (9.27210993082233E+016,9.27282993082337E+016),
         (1.19429200349407E+017,1.19436400349418E+017)
        ]

FVBS6 = [(6.70902889090192E+015,6.71532889090465E+015),
         (1.36233982073171E+016,1.36305982073196E+016),
         (5.20266982526602E+016,5.20274182526611E+016),
         (9.27210993082233E+016,9.27282993082337E+016),
         (1.19429200349407E+017,1.19436400349418E+017)
        ]

FVBS7 = [(6.70902889090192E+015,6.71532889090465E+015),
         (1.36233982073171E+016,1.36305982073196E+016),
         (5.21994982528778E+016,5.22002182528788E+016),
         (9.27210993082233E+016,9.27282993082337E+016),
         (1.19429200349407E+017,1.19436400349418E+017)
        ]

FVBS = [FVBS1,FVBS2,FVBS3,FVBS4,FVBS5,FVBS6,FVBS7]

In [6]:
def clean_files(files, periods):
    # do the cleaning - get rid of all files within the listed time periods
    to_remove = []  # files to remove later
    for period in periods:
        tstart = period[0]
        tstop = period[1]
        for file in files:
            time = getobmt(file)
            if (time>=tstart) and (time<=tstop):
                to_remove.append(file)
    for file in to_remove:
        files.remove(file)

In [7]:
# write 14 files

for ifov in range(2):
    for irow in range(7):
        data = filenames[ifov][irow]
        clean_files(data,OBMT_CI+FVBS[irow])
        outname = "filenames_sm{}_row_{}.txt".format(ifov+1,irow+1)
        
        file = open(outname, 'w')

        for path in data:
            file.write("%s" % path)

        file.close()

In [11]:
# and one for each fov
for ifov in range(2):
    fov_files = []
    for irow in range(7):
        data = filenames[ifov][irow]
        clean_files(data,OBMT_CI+FVBS[irow])
        fov_files += data
    fov_files.sort(key=getobmt)    
    outname = "filenames_sm{}_clean.txt".format(ifov+1)    
    file = open(outname, 'w')
    for path in fov_files:
        file.write("%s" % path)
    file.close()
